Generate a Audit Trail test file

In [ ]:
#Define User Variable
import pandas as pd

dataBase = pd.DataFrame(columns = ['Timestamp', 'User ID', 'Object ID', 'Description', 'Comments'])

atStartDate = '01.05.2020 6:00:00'
atEndDate = '01.05.2020 15:00:00' # alternativ könnte hier +8h stehen
maxAtEntries = 10

userDB = pd.DataFrame( columns = ['User ID', 'Probablility'] )

userDB['User ID'] = ['Admin', 'System']
userDB['Probablility'] = [0.2, 0.1]


In [ ]:
#Define Users
userEx = pd.DataFrame( columns = ['User ID', 'Probablility'] )

userEx['User ID'] = ['pharma/de01234', 'pharma/I047874', 'pharma/I047874', 'pharma/I984321']


#Define probablility

#equal probability
val = (1 - userDB[ userDB.columns[1] ].sum()) / userEx.shape[0]
userEx['Probablility'] = [val] * userEx.shape[0]
#userDB = userDB.append( {'User ID': user, 'Probablility': prop }, ignore_index=True )

userDB = pd.concat([userDB, userEx], ignore_index = True)

userDB

In [3]:
userIDs = userDB['User ID'].to_list()
#bring the description dict into the userDB as well
#system and user extension separat

descriptions = {
    userIDs[0]: [['Wechsel in die Betriebsart Online', 'Wechsel in die Betriebsart Offline', 'Anwendung wird gestartet', 'Anwendung wird beendet'],
              [                                0.2,                                  0.2,                        0.2,                     0.4]],
    userIDs[1]: [['Benutzer # wurde hinzugefügt', 'Benutzer # wurde entfernt', 'Benutzer # ist keiner Gruppe zugeordnet', 'Benutzer # ist einer neuen Gruppe zugeordnet'],
               [                           0.1,                         0.1,                                       0.4,                                           0.4]],  
    userIDs[2]: [['Umfang # wurde geändert', 'Schwellwert wurde geändert', 'Wert wurde gelöscht', 'Hambeuldu'],
              [                        0.1,                        0.1,                   0.4,        0.4]],
}
    
descriptions[userIDs[3]] = descriptions[userIDs[2]]
descriptions[userIDs[4]] = descriptions[userIDs[2]]
descriptions[userIDs[5]] = descriptions[userIDs[2]]

In [4]:
#User login and logof entries
logIn = 'An'
logOff = 'Ab'

logEntryDic = {
    'UserId': userIDs[1], 
    'ObjectId': 'Benutzerverwaltung',
    'Description': 'meldung von Benutzer # war erfolgreich.' 
}

In [5]:
#3. enhance database entries
userList = []
objectList = []
descriptionList = []
commentsList = []

In [6]:
#helping functions
def logEntry(logPostFix, user):
    userList.append(logEntryDic['UserId'])
    objectList.append(logEntryDic['ObjectId'])
    descriptionList.append(logPostFix + re.sub('#', user, logEntryDic['Description']))
    
def userEntry(user):
    userList.append(user)
    objectList.append('ObjectID for user ' + user )#random user related object ID
    
    ds = np.random.choice(descriptions[user][0], 1, p=descriptions[user][1])                          
    descriptionList.append(ds[0])# random user related description ID
    

In [7]:
#Generate random values
#1. numbers of audit trail entries 
import numpy as np # import random insteed
import re #regex 

nOfAtEntries = np.random.choice(maxAtEntries, 1)
#userEntries = np.random.choice(userIDs, nOfAtEntries, p=userProp)
userEntries = np.random.choice(userDB['User ID'].to_list(), nOfAtEntries, p=userDB['Probablility'].to_list())
#plot the distribution of users

In [8]:
#3. enhance database entries
loggedUser = None
newUser = None


for user in userEntries:
    if 'pharma/' not in user:
        if loggedUser:
            loggedUser = None
            logEntry(logOff, user)
        userEntry(user)
    
    if 'pharma/' in user:
        newUser = user
        while newUser:
            if not loggedUser:
                loggedUser = user
                logEntry(logIn, user) 
            if loggedUser == user:
                userEntry(user)
                newUser = None
            else:
                logEntry(logOff, user)
                loggedUser = None
                newUser = user
            
if loggedUser:
    logEntry(logOff, user)
    loggedUser = None

In [9]:
#2. Random timestamps for each entry
entries = np.random.choice(pd.date_range(atStartDate, atEndDate, freq='S'), 
                           len(userList), 
                           replace=False)

In [10]:
dataBase['Timestamp'] = pd.to_datetime(np.sort(entries))
dataBase['User ID'] = userList
dataBase['Object ID'] = objectList
dataBase['Description'] = descriptionList

dataBase.to_csv('auditTrail.csv')
dataBase

,Timestamp,User ID,Object ID,Description,Comments
0,2020-01-05 07:38:54,System,Benutzerverwaltung,Anmeldung von Benutzer pharma/I047874 war erfo...,NaN
1,2020-01-05 08:30:48,pharma/I047874,ObjectID for user pharma/I047874,Wert wurde gelöscht,NaN
2,2020-01-05 09:51:58,System,Benutzerverwaltung,Abmeldung von Benutzer pharma/I984321 war erfo...,NaN
3,2020-01-05 10:17:02,System,Benutzerverwaltung,Anmeldung von Benutzer pharma/I984321 war erfo...,NaN
4,2020-01-05 11:06:42,pharma/I984321,ObjectID for user pharma/I984321,Wert wurde gelöscht,NaN
5,2020-01-05 14:31:17,System,Benutzerverwaltung,Abmeldung von Benutzer pharma/I984321 war erfo...,NaN
